# Example Ensembles: Australian Credit Card Assessment

In this example it will be compared 3 classifiers:
- Multilayer perceptron.
- Ensemble trained with Negative Correlation.
- Ensemble trained with Correntropy.

## Data

This data base concerns credit card applications. All attribute names and values have been changed to meaningless symbols to protect confidentiality of the data. This dataset is interesting because there is a good mix of attributes -- continuous, nominal with small numbers of values, and nominal with larger numbers of values. There are also a few missing values.
  
- Number of Instances: 690
- Number of Attributes: 14 + class attribute
- Attribute Information:   THERE ARE 6 NUMERICAL AND 8 CATEGORICAL ATTRIBUTES.
- Exist 2 classes and its distribution is:
    * +: 307 (44.5%)    CLASS 2
    * -: 383 (55.5%)    CLASS 1
- there are 6 numerical and 8 categorical attributes. the labels have been changed for the convenience of the statistical algorithms. For example, attribute 4 originally had 3 labels p,g,gg and these have been changed to labels 1,2,3.

- Missing Attribute Values: 37 cases (5%) HAD one or more missing values. The missing values from particular attributes WERE:
    * A1:  12
    * A2:  12
    * A4:   6
    * A5:   6
    * A6:   9
    * A7:   9
    * A14: 13
 
  These were replaced by the mode of the attribute (categorical) and mean of the attribute (continuous)


In [ ]:
import theano
import theano.tensor as T
import numpy as np
from theano.sandbox import cuda

theano.config.floatX = 'float32'
cuda.use('gpu')
theano.config.compute_test_value = 'off'

path_data = r'data\australian.dat'
data = np.genfromtxt(path_data, delimiter=' ')

classes_names = np.asarray(['clase 1', 'clase 2'], dtype='<U10')
data_input    = np.asarray(data[:, 0:-1], dtype=theano.config.floatX)
data_target   = classes_names[np.asarray(data[:, -1:], dtype=int)]

## Training MLP



In [ ]:
import time
from libml.models.sequential import Sequential
from libml.layers.dense import Dense
from libml.utils import *

from sklearn.cross_validation import ShuffleSplit

mlp = Sequential(classes_names, "classifier", "mlp")
mlp.add_layer(Dense(n_input=data_input.shape[1], n_output=10, activation=T.nnet.sigmoid))
mlp.add_layer(Dense(n_output=len(classes_names), activation=T.nnet.sigmoid))
mlp.append_cost(mse)
mlp.set_update(sgd_momentum, learning_rate=0.1, momentum_rate=0.9)
mlp.compile()

folds = 3
sss = ShuffleSplit(data_input.shape[0], n_iter=folds, test_size=None, train_size=0.75, random_state=0)
max_epoch = 250
validation_jump = 5

metrics_mlp = ClassifierMetrics(mlp)

for i, (train_set, test_set) in enumerate(sss):                      
    # data train and test
    input_train = data_input[train_set]
    input_test = data_input[test_set]
    target_train = data_target[train_set]
    target_test = data_target[test_set]

    # training
    tic = time.time()
    metric = mlp.fit(input_train, target_train,
                                max_epoch=max_epoch, batch_size=32,
                                validation_jump=validation_jump, early_stop_th=4)
    toc = time.time()

    # Compute metrics
    metrics_mlp.append_prediction(target_test, mlp.predict(input_test))
    metrics_mlp.append_metric(metric)
    
    # Reset parameters
    mlp.reset()

    print("%d Elapsed time [s]: %f" % (i, toc - tic))

print("FINISHED!")

## Results MLP

In [ ]:
%matplotlib inline
import matplotlib.pylab as plt

metrics_mlp.plot_confusion_matrix()
metrics_mlp.plot_cost(max_epoch, "Cost training MLP")
metrics_mlp.plot_score(max_epoch, "Accuracy training MLP")

plt.tight_layout()

## Training Ensemble - Negative Correlation (Xin Yao, 1999)

In [ ]:
from libml.ensemble.ensemblemodel import EnsembleModel

# Create Ensemble
ensemble = EnsembleModel()

# Create models for ensemble
for i in range(4):
    net = Sequential(classes_names, "classifier", "net%d_ens_nc" % i)
    net.add_layer(Dense(n_input=data_input.shape[1], n_output=10, activation=T.nnet.sigmoid))
    net.add_layer(Dense(n_output=len(classes_names), activation=T.nnet.sigmoid))
    net.append_cost(mse)
    net.set_update(sgd_momentum, learning_rate=0.1, momentum_rate=0.9)
    ensemble.append_model(net)

ensemble.add_cost_ensemble(fun_cost=neg_corr, lamb_neg_corr=1.0)  # adds neg correlation in all models
ensemble.compile()

metrics_ensemble = EnsembleClassifierMetrics(ensemble)

for i, (train_set, test_set) in enumerate(sss):                      
    # data train and test
    input_train = data_input[train_set]
    input_test = data_input[test_set]
    target_train = data_target[train_set]
    target_test = data_target[test_set]

    # training
    tic = time.time()
    metrics = ensemble.fit(input_train, target_train,
                                max_epoch=max_epoch, batch_size=32,
                                validation_jump=validation_jump, early_stop_th=4)
    toc = time.time()

    # Compute metrics
    metrics_ensemble.append_prediction(target_test, ensemble.predict(input_test))
    metrics_ensemble.append_metric(metrics)
    
    # Reset parameters
    ensemble.reset()

    print("%d Elapsed time [s]: %f" % (i, toc - tic))

print("FINISHED!")

## Results Ensemble - Negative Correlation

In [ ]:
%matplotlib inline
import matplotlib.pylab as plt

metrics_ensemble.plot_confusion_matrix()
metrics_ensemble.plot_cost(max_epoch, "Cost training Ensemble-NC")
metrics_ensemble.plot_cost_models(max_epoch, "Cost training Ensemble-NC per models")
metrics_ensemble.plot_score(max_epoch, "Accuracy training Ensemble-NC")
metrics_ensemble.plot_score(max_epoch, "Accuracy training Ensemble-NC per models")

plt.tight_layout()